# Clean data from video annotator

Create a script to clean data from video annotator

install python video annotator: http://pythonvideoannotator.readthedocs.io/en/master/

## Run Video Annotator in Terminal

### for windows:
- activate videoannotator
- python -m pythonvideoannotator

### for mac:
- source activate videoannotator
- python -m pythonvideoannotator

## List of tasks
self.taskList = np.array(['Shaking', 'Standing', 'Walking', 'Walking while counting', 'Finger to nose--right hand', 
                          'Finger to nose--left hand', 'Alternating right hand movements', 
                          'Alternating left hand movements', 'Sit to stand', 'Drawing on a paper', 
                          'Typing on a computer keyboard', 'Assembling nuts and bolts', 'Taking a glass of water',
                          'Organizing sheets in a folder', 'Folding towels', 'Sitting'])

Shaking
Standing
Walking
Walking while counting
Finger to nose--right hand
Finger to nose--left hand
Alternating right hand movements
Alternating left hand movements
Sit to stand
Drawing on a paper
Typing on a computer keyboard
Assembling nuts and bolts
Taking a glass of water
Organizing sheets in a folder
Folding towels
Sitting

In [ ]:
# For Joe's computercd Y:

In [1]:
# Importing the Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import datetime as dt
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
#import time
#import copy

In [2]:
# specify path
#path = r'C:\Users\lu-rt&o\Documents\GitHub\CIS-PD MUSC\CIS-PD-MUSC'
#path = r'C:\Users\lu-rt&o\pythonVideoAnnotator'
path = r'Y:\CIS-PD Videos\Cisnwh8'

## Videoannotator rows correspond to video cycle number

In [ ]:
# add subject id column

In [ ]:
def dataclean(file):    
    colnames = ['subject code', '1', 'start time', 'stop time', 'activity', '2', 'cycle']
    selectedcol= ['subject code','start time', 'stop time', 'activity', 'cycle']
    #df = pd.read_csv(os.path.join(path, 'cycle1.csv'), names=colnames, usecols=selectedcol)
    filename = os.path.join(path, file) # ie. file = 'cycle1.csv'
    df = pd.read_csv(filename, names=colnames, usecols=selectedcol)
    df = df.dropna()
    df['cycle'] += 1
    df['stop time'] = pd.to_numeric(df['stop time'], downcast='integer')
    df['cycle'] = pd.to_numeric(df['cycle'], downcast='integer')
    df['subject code'] = 1030
    print(df.head(3))
    
    return df

In [ ]:
df1 = dataclean('cycle1.csv')
df2 = dataclean('cycle2.csv')
df3 = dataclean('cycle3incomplete.csv')
df4 = dataclean('cycle36.csv')

In [ ]:
frames = [df1, df2, df3, df4]
df = pd.concat(frames, ignore_index=True)

In [ ]:
print(df.head(5))
print(df.tail(5))

In [ ]:
filename2 = os.path.join(path, '1030annotated.csv')
with open(filename2,'wb') as f:
    df.to_csv(filename2, sep=',')

# Load and clean annotated file

In [ ]:
filename = os.path.join(path, '1030annotated.csv')
df = pd.read_csv(filename)
df = df.rename(index=str, columns={'Unnamed: 6': 'start time min', 'Unnamed: 7':'stop time min',
                             'Unnamed: 8': 'shortname'})
df = df.drop(columns = ['Unnamed: 0'])

# add column of activity abbreviated name
# omit 'Shaking' activity
x = np.array(['Stndg', 'Wlkg', 'WlkgCnt', 'FtnR', 'FtnL', 'RamR', 'RamL', 'SitStand', 
              'Drwg', 'Typg', 'NtsBts', 'Drnkg', 'Sheets', 'Fldg', 'Sitng'])
activity = ('Stndg', 'Wlkg', 'WlkgCnt', 'FtnR', 'FtnL', 'RamR', 'RamL', 'SitStand', 
              'Drwg', 'Typg', 'NtsBts', 'Drnkg', 'Sheets', 'Fldg', 'Sitng')
df['shortname'] = np.array(activity*6)

In [ ]:
def frame_to_sec(frames,fps=30):
    """Convert frames with fps sampling rate to time in total sec."""
    seconds = int((frames / (60 * fps) % 60)*60)
    return seconds

In [ ]:
# skip this
def frame_to_time(frames,fps=30):
    """Convert frames with fps sampling rate to time in min and sec."""
    # split frames into times
    hours = int(frames / (3600 * fps))
    minutes = int(frames / (60 * fps) % 60)
    seconds = int(frames / fps % 60)
    milliseconds = frames / fps
    #milliseconds = int(((frames / fps) - int(frames / fps % 60)) * 10)
    
    # returns a string object
    return "%02d:%02d:%02d:%03d" % (hours,minutes, seconds, milliseconds)
    
    # convert to time object??

# Created dataframe with fps converted to time in string

In [ ]:
# Create time column by applying function to start and stop time columns 
df['start time min'] = df['start time'].apply(frame_to_time)
df['stop time min'] = df['stop time'].apply(frame_to_time)
df.head(5)

In [ ]:
# need to convert start/stop time type from str to time
x = df['start time min'][0]
a2 = dt.datetime.strptime(x,"%H:%M:%S:%f").time()
print(a2)

In [ ]:
# clip video trial
video_full = 'Cisnwh8_cycle1.mp4'
activity = 'Stndg'
targetname = activity + '.mp4'
t1 = df['start time min'][0] #1118
t1conv = dt.datetime.strptime(t1,"%H:%M:%S:%f").time()
t2 = df['stop time min'][0] #2145
t2conv = dt.datetime.strptime(t2,"%H:%M:%S:%f").time()
ffmpeg_extract_subclip(video_full, t1conv, t2conv, targetname=targetname)

In [ ]:
# clip video trial
video_full = os.path.join(path, 'Cisnwh8_cycle1.mp4')
activity = 'Stndg'
targetname = activity + '.mp4'
t1 = 37 #1118
t2 = 71 #2145
ffmpeg_extract_subclip(video_full, t1, t2, targetname=targetname)

In [ ]:
filename = os.path.join(path, '1030_time_str.csv')
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',')

# Convert fps to time in sec

In [ ]:
df['start time min'] = df['start time'].apply(frame_to_sec)
df['stop time min'] = df['stop time'].apply(frame_to_sec)
df.head(5)

In [ ]:
filename = os.path.join(path, '1030_time_in_sec.csv')
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',')

# Clip all activities and save to cycle subfolder

In [ ]:
# convert to column from int to str type
df['cycle'] = df['cycle'].apply(str)

In [ ]:
# Loop through and clip each activity to corresponding cycle video
for i in range(len(df)):
    cycleindex = df['cycle'][i]
    video_name = 'Cisnwh8_cycle' + cycleindex + '.mp4'
    video_full = os.path.join(path, video_name)
    t1 = df['start time min'][i]
    t2 = df['stop time min'][i]
    activity = df['shortname'][i]
    # path_for_target = os.path.join(path,'\cycle' + cycleindex)
    path_for_target = path + '\cycle' + cycleindex + '\\'
    targetname = path_for_target + activity + '.mp4'
    print(activity, t1, t2, targetname)
    
    # clip activity video
    ffmpeg_extract_subclip(video_full, t1, t2, targetname=targetname)

# Adjust clip times and reclip videos

In [20]:
filename = os.path.join(path, '1030.csv')
df2 = pd.read_csv(filename)

In [ ]:
# if it doesn't work...
df2['stop time min'][4] = df2['stop time min'][4] + 2
df2['stop time min'][5] = df2['stop time min'][5] + 1
df2['start time min'][6] = df2['start time min'][6] + 1
df2['stop time min'][7] = df2['stop time min'][7] + 2
df2['stop time min'][11] = df2['stop time min'][11] + 2
df2['stop time min'][12] = df2['stop time min'][12] + 2
df2['stop time min'][18] = df2['stop time min'][18] + 1
df2['start time min'][19] = df2['start time min'][19] - 1
df2['stop time min'][27] = df2['stop time min'][27] + 1
df2['stop time min'][37] = df2['stop time min'][37] + 1
df2['stop time min'][38] = df2['stop time min'][38] + 1
df2['stop time min'][40] = df2['stop time min'][40] + 1
df2['stop time min'][43] = df2['stop time min'][43] + 2
df2['stop time min'][51] = df2['stop time min'][51] + 1
df2['stop time min'][52] = df2['stop time min'][52] + 2
df2['stop time min'][53] = df2['stop time min'][53] + 1
df2['stop time min'][54] = df2['stop time min'][54] + 1
df2['stop time min'][55] = df2['stop time min'][55] + 1
df2['stop time min'][56] = df2['stop time min'][56] + 2
df2['stop time min'][57] = df2['stop time min'][57] + 1
df2['stop time min'][67] = df2['stop time min'][67] + 1
df2['stop time min'][68] = df2['stop time min'][68] + 1
df2['stop time min'][69] = df2['stop time min'][69] + 1
df2['stop time min'][82] = df2['stop time min'][82] + 2
df2['stop time min'][83] = df2['stop time min'][83] + 1
df2['stop time min'][84] = df2['stop time min'][84] + 1
df2['stop time min'][85] = df2['stop time min'][85] + 1

In [26]:
df2['stop time min'][53] = df2['stop time min'][53] + 1
df2['stop time min'][68] = df2['stop time min'][68] + 1

362 316
363 317


C:\ProgramData\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
# Loop through and clip each activity to corresponding cycle video
df2['cycle'] = df2['cycle'].apply(str)

for i in range(len(df2)):
    cycleindex = df2['cycle'][i]
    video_name = 'Cisnwh8_cycle' + cycleindex + '.mp4'
    video_full = os.path.join(path, video_name)
    t1 = df2['start time min'][i]
    t2 = df2['stop time min'][i]
    activity = df2['shortname'][i]
    # path_for_target = os.path.join(path,'\cycle' + cycleindex)
    path_for_target = path + '\cycle' + cycleindex + '\\'
    targetname = path_for_target + activity + '.mp4'
    print(activity, t1, t2, targetname)
    
    # clip activity video
    ffmpeg_extract_subclip(video_full, t1, t2, targetname=targetname)

Stndg 37 71 Y:\CIS-PD Videos\Cisnwh8\cycle1\Stndg.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle1.mp4 -ss 37.00 -t 34.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle1\Stndg.mp4
... command successful.
Wlkg 77 111 Y:\CIS-PD Videos\Cisnwh8\cycle1\Wlkg.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle1.mp4 -ss 77.00 -t 34.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle1\Wlkg.mp4
... command successful.
WlkgCnt 120 160 Y:\CIS-PD Videos\Cisnwh8\cycle1\WlkgCnt.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle1.mp4 -ss 120.00 -t 40.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle1\WlkgCnt.mp4
... command successful.
FtnR 194 217 Y:\CIS-PD Videos\Cisnwh8\cycle1\FtnR.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i 

>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle2.mp4 -ss 587.00 -t 35.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle2\Fldg.mp4
... command successful.
Sitng 627 661 Y:\CIS-PD Videos\Cisnwh8\cycle2\Sitng.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle2.mp4 -ss 627.00 -t 34.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle2\Sitng.mp4
... command successful.
Stndg 40 74 Y:\CIS-PD Videos\Cisnwh8\cycle3\Stndg.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle3.mp4 -ss 40.00 -t 34.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle3\Stndg.mp4
... command successful.
Wlkg 79 119 Y:\CIS-PD Videos\Cisnwh8\cycle3\Wlkg.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle3.mp4 -ss 79.00 -t 40.00 -vcodec copy -ac

>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle4.mp4 -ss 464.00 -t 31.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle4\Drnkg.mp4
... command successful.
Sheets 516 565 Y:\CIS-PD Videos\Cisnwh8\cycle4\Sheets.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle4.mp4 -ss 516.00 -t 49.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle4\Sheets.mp4
... command successful.
Fldg 578 611 Y:\CIS-PD Videos\Cisnwh8\cycle4\Fldg.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle4.mp4 -ss 578.00 -t 33.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle4\Fldg.mp4
... command successful.
Sitng 614 669 Y:\CIS-PD Videos\Cisnwh8\cycle4\Sitng.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle4.mp4 -ss 614.00 -t 55.00 -vcodec 

>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle6.mp4 -ss 328.00 -t 39.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle6\Typg.mp4
... command successful.
NtsBts 378 426 Y:\CIS-PD Videos\Cisnwh8\cycle6\NtsBts.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle6.mp4 -ss 378.00 -t 48.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle6\NtsBts.mp4
... command successful.
Drnkg 433 462 Y:\CIS-PD Videos\Cisnwh8\cycle6\Drnkg.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle6.mp4 -ss 433.00 -t 29.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle6\Drnkg.mp4
... command successful.
Sheets 488 533 Y:\CIS-PD Videos\Cisnwh8\cycle6\Sheets.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle6.mp4 -ss 488.00 -t 45.00 -vco

In [28]:
i=53
cycleindex = df2['cycle'][i]
video_name = 'Cisnwh8_cycle' + cycleindex + '.mp4'
video_full = os.path.join(path, video_name)
t1 = df2['start time min'][i]
t2 = df2['stop time min'][i]
activity = 'Drwg'
# path_for_target = os.path.join(path,'\cycle' + cycleindex)
path_for_target = path + '\cycle' + cycleindex + '\\'
targetname = path_for_target + activity + '.mp4'
print(activity, t1, t2, targetname)

# clip activity video
ffmpeg_extract_subclip(video_full, t1, t2, targetname=targetname)

Drwg 326 363 Y:\CIS-PD Videos\Cisnwh8\cycle4\Drwg.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle4.mp4 -ss 326.00 -t 37.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle4\Drwg.mp4
... command successful.


In [30]:
i=68
cycleindex = df2['cycle'][i]
video_name = 'Cisnwh8_cycle' + cycleindex + '.mp4'
video_full = os.path.join(path, video_name)
t1 = df2['start time min'][i]
t2 = df2['stop time min'][i]
activity = 'Drwg'
# path_for_target = os.path.join(path,'\cycle' + cycleindex)
path_for_target = path + '\cycle' + cycleindex + '\\'
targetname = path_for_target + activity + '.mp4'
print(activity, t1, t2, targetname)

# clip activity video
ffmpeg_extract_subclip(video_full, t1, t2, targetname=targetname)

Drwg 280 317 Y:\CIS-PD Videos\Cisnwh8\cycle5\Drwg.mp4

[MoviePy] Running:
>>> C:\ProgramData\anaconda3\Library\bin\ffmpeg.exe -y -i Y:\CIS-PD Videos\Cisnwh8\Cisnwh8_cycle5.mp4 -ss 280.00 -t 37.00 -vcodec copy -acodec copy Y:\CIS-PD Videos\Cisnwh8\cycle5\Drwg.mp4
... command successful.


In [32]:
df2 = df2.rename(index=str, columns={'start time min': 'start time sec', 
                                     'stop time min':'stop time sec',})

In [33]:
df2.head(2)

,Unnamed: 0,subject code,start time,stop time,activity,cycle,start time sec,stop time sec,shortname,Unnamed: 9,Unnamed: 10
0,0,1030,1118,2145,Standing,1,37,71,Stndg,NaN,NaN
1,1,1030,2315,3333,Walking,1,77,111,Wlkg,NaN,NaN


In [34]:
filename = os.path.join(path, '1030.csv')
with open(filename,'wb') as f:
    df2.to_csv(filename, sep=',')